### AirBnb listings dataset cleaning
Source: http://insideairbnb.com/get-the-data.html - listings.csv.gz - Detailed Listings data for Barcelona

The specific file used is the one related to the 14th of August 2018. The reason why a 2018 dataset was used is because the analysis will be carried out in relation to Barcelona's average rental prices as provided by the Ajuntament of Barcelona, which are released every year. For obvious reasons the complete data available currently stops at 2018.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as pyplot

In [2]:
airbnb_full = pd.read_csv("/home/emanuele/Desktop/IronHack/Projects/Project-Week-2-Barcelona/your-project/listings_jun.csv")

Given the sheer number of attributes in the listings.csv file it is preferrable to list the columns instead of using head() or describe()

In [3]:
list(airbnb_full.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


The columns that will be kept will be related only the data strictly needed for the analysis: 'id', for matters of indexing; 'price', since our analysis will be fully related to the prices of the listings in comparison to rental prices; the columns related to neighbourhood data; 'accomodates', which stores the maximum number of people allowed; 'availability_30', which can be used to determine the percentage of booked days in the 30 days prior to the 14th of August 2018 to better relate the listing price to a rental price, which of course assumes a "booked" percentage of 100%. We will also be using 'room_type' initially to limit the data to listings related to the Entire house, exluding Private and Shared rooms since we assume the prices to be skewed. 

Given the presence of 3 different neighbourhood attributes and the possibility of carrying out a more detailed process that doesn't stop at a per-district analysis but goes deeper to a per-neighbourhood one (e.g.: El Clot, La Verneda y La Pau, etc. instead of simply the whole Sant Martì district) we will first check a sample for the aforementioned columns to determine which ones, if not all, are useful in our case.

We will also be checking the different types of listings through the 'room_type' column.

In [4]:
airbnb_full.loc[:,('neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed')].sample(30)

,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed
14080,El Gòtic,el Barri Gòtic,Ciutat Vella
4184,Dreta de l'Eixample,la Dreta de l'Eixample,Eixample
1115,La Nova Esquerra de l'Eixample,la Nova Esquerra de l'Eixample,Eixample
303,Sants-Montjuïc,Sants - Badal,Sants-Montjuïc
197,Sants-Montjuïc,Sants,Sants-Montjuïc
12896,El Raval,el Raval,Ciutat Vella
16541,La Nova Esquerra de l'Eixample,la Nova Esquerra de l'Eixample,Eixample
4167,El Raval,el Raval,Ciutat Vella
6039,Sant Gervasi - Galvany,Sant Gervasi - Galvany,Sarrià-Sant Gervasi
8443,La Sagrada Família,la Sagrada Família,Eixample


In [5]:
airbnb_full['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

We decided to use the cleansed columns given the absence of NaN values. We also confirmed that 'neighborhood_group_cleansed' refers to a district while 'neighborhood_cleansed' refers to the smaller "Barrios".

That said, we proceed with creating a new DataFrame including Entire homes only (since they relate better to a rental house agreement when compared to a Private or Shared room) and then dropping all the unneeded columns.

In [6]:
airbnb_full = airbnb_full[airbnb_full['room_type'] == 'Entire home/apt']

In [7]:
airbnb_full = airbnb_full.loc[:,('id', 'neighbourhood_group_cleansed', 'price', 'neighbourhood_cleansed' ,'accommodates','availability_60','last_review','minimum_nights')]

We decided to cutoff all homes with an available accomodation of more than 6 people to try and limit the effect that apartments renovated as vacation homes (i.e. with more beds or sleeping spaces that a normal house of the same size would accomodate, and therefore more people) will not severely influence our final result

In [8]:
airbnb_full = airbnb_full[airbnb_full['accommodates'] <= 6]

The data related to each listing's availability (which will be used in the analysis to weight the listing price when compared to a traditional monthly rental as accurately as possible) is, as explained in the source website, parsed from the calendar on the AirBnb website. This calendar **DOES NOT** discern unavailable days between those caused by a pre-existing booking or a simple choice to make the listing unavailable by the host.

To try and clean the data of listings that have been inactive for a long time as of 2018-08-14 we will be filtering out of all the listings with a last review that dates **more than 2 months before** the date of the parsing. Given the sheer number of observations available we believe that even a timeframe of 2 only months, at least regarding the review, will offer enough data for the analysis to be relevant.

The next cell's purpose is to check the latest date that a review was added, so that the following slice will respect the 2-months cutoff timeframe.

In [9]:
airbnb_full.loc[:,('id','last_review')].sort_values('last_review', ascending=False).head()

,id,last_review
6166,11685016,2018-06-08
11679,20431737,2018-06-08
3553,5392353,2018-06-08
11271,19957204,2018-06-08
15584,24722608,2018-06-08


Slicing the dataframe again. The slice is from the last review date, 2018-08-14, to 2 months prior, 2018-06-14. Also dropping NaN values.

In [10]:
airbnb_full = airbnb_full[(airbnb_full['last_review'] <= '2018-06-08') & (airbnb_full['last_review'] >= '2018-04-08')]
airbnb_full = airbnb_full[(airbnb_full['last_review'] != np.nan)]

We can now drop the 'last_review' column since it fulfilled its purpose.

In [11]:
airbnb_full = airbnb_full.drop('last_review', axis=1)

Checking prices for possible outliers.

In [12]:
airbnb_full.sort_values('price', ascending=False).head(50)

,id,neighbourhood_group_cleansed,price,neighbourhood_cleansed,accommodates,availability_60,minimum_nights
546,721470,Eixample,$999.00,el Fort Pienc,2,58,3
550,721509,Eixample,$999.00,la Dreta de l'Eixample,4,38,3
13079,22033469,Eixample,$999.00,la Nova Esquerra de l'Eixample,6,0,120
4204,6819336,Eixample,$99.00,la Dreta de l'Eixample,5,11,4
3020,4213133,Ciutat Vella,$99.00,el Raval,6,1,2
5233,9237132,Ciutat Vella,$99.00,el Raval,4,20,2
8304,15893888,Eixample,$99.00,Sant Antoni,6,31,4
4008,6507867,Eixample,$99.00,la Sagrada Família,4,51,2
4280,6962648,Eixample,$99.00,el Fort Pienc,3,39,4
925,1010589,Sants-Montjuïc,$99.00,el Poble Sec,6,9,3


Given the sorting the prices are currently saved as a string. Since they are shown with a "$" sign we will need to drop it to convert the type to int.

In [13]:
airbnb_full['price'] = airbnb_full['price'].replace("\$", "", regex=True)
airbnb_full['price'] = airbnb_full['price'].replace(",", "", regex=True)

Converting to int

In [14]:
airbnb_full['price'] = pd.to_numeric(airbnb_full['price'])

Checking for outliers on high prices.

In [15]:
airbnb_full.sort_values('price', ascending=False)

,id,neighbourhood_group_cleansed,price,neighbourhood_cleansed,accommodates,availability_60,minimum_nights
8655,16597057,Les Corts,3000.0,la Maternitat i Sant Ramon,2,28,2
8660,16607374,Les Corts,3000.0,la Maternitat i Sant Ramon,2,26,2
8661,16607759,Les Corts,3000.0,la Maternitat i Sant Ramon,2,36,2
13753,22978545,Horta-Guinardó,1180.0,el Guinardó,5,23,5
13563,22741727,Gràcia,1000.0,la Vila de Gràcia,2,28,2
13560,22741274,Gràcia,1000.0,la Vila de Gràcia,5,12,2
6486,12534863,Sants-Montjuïc,1000.0,la Font de la Guatlla,6,50,2
3807,6034571,Sant Martí,1000.0,el Poblenou,2,30,3
3808,6035889,Sant Martí,1000.0,el Poblenou,6,43,3
8580,16436541,Sant Martí,1000.0,el Poblenou,5,41,2


Creating a new column for "revenue", which will simply be the price multiplied by the actual number of booked days (calculated as 30 - availability_30) to estimate a monthly revenue and compare it with the rental prices.

In [16]:
map_func = lambda x: round((x['price'] * (60 - x['availability_60']))/2, 2)
airbnb_full['revenue'] = airbnb_full.apply(map_func, axis = 1)

In [17]:
price_by_district = round(airbnb_full.groupby(['neighbourhood_group_cleansed']).mean(),2)



In [18]:
price_by_district

,id,price,accommodates,availability_60,minimum_nights,revenue
neighbourhood_group_cleansed,,,,,,
Ciutat Vella,15032265.61,117.43,3.84,18.11,4.59,2377.71
Eixample,10664289.50,156.27,4.72,19.57,2.86,3017.02
Gràcia,9699926.23,135.86,4.23,18.49,3.18,2752.22
Horta-Guinardó,12189816.85,157.97,4.25,17.32,2.57,2811.40
Les Corts,11242483.47,213.90,4.04,24.10,2.14,3378.92
Nou Barris,9425841.33,81.11,3.78,17.33,2.78,1638.33
Sant Andreu,9365560.50,80.86,4.36,10.18,2.64,2023.45
Sant Martí,11273150.72,134.58,4.63,18.09,2.91,2552.73
Sants-Montjuïc,10012109.15,103.79,4.36,18.79,3.04,2049.39


In [19]:
price_by_district.to_csv('price_by_district.csv')



In [ ]:
for index in range(0, len(test)):
    if index == 0:
        if test.iloc[index] < 10:
            test.iloc[index] = np.nan
    else:
        if test.iloc[index] < 10:
            if not test.iloc[index - 1] >= 10:
                test.iloc[index] = np.nan
            else:
                test.iloc[index] = test.iloc[index - 1]
        
            
    
test